## 소분류에서 공통으로 많이 쓰이는 키워드 제거

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 로드

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/GBT Hackerton/data/train_df_translate_del.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GBT Hackerton/data/test_df_translate_del.csv')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [5]:
train_data.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


In [6]:
test_data.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


### 소분류별 분석

각 데이터별 키워드 개수

In [7]:
# 각 데이터의 키워드 개수를 세고, 평균, 최대, 최소값을 계산
def calculate_keyword_statistics(df):
    # 키워드 개수 계산
    df['키워드 개수'] = df['키워드'].apply(lambda x: len(x.split(',')))

    # 평균, 최대, 최소값 계산
    avg_keywords = df['키워드 개수'].mean()
    max_keywords = df['키워드 개수'].max()
    min_keywords = df['키워드 개수'].min()

    print("\n각 데이터의 키워드 개수 통계:")
    print("평균 키워드 개수: {:.2f}".format(avg_keywords))
    print("최대 키워드 개수: {}".format(max_keywords))
    print("최소 키워드 개수: {}".format(min_keywords))

In [8]:
calculate_keyword_statistics(train_data)


각 데이터의 키워드 개수 통계:
평균 키워드 개수: 168.61
최대 키워드 개수: 2669
최소 키워드 개수: 4


In [10]:
calculate_keyword_statistics(test_data)


각 데이터의 키워드 개수 통계:
평균 키워드 개수: 170.46
최대 키워드 개수: 2549
최소 키워드 개수: 5


소분류별 많이 등장한 공통 키워드 제거

In [9]:
train_data['분류'].nunique()

56

In [11]:
# 많이 등장한 공통 키워드 추출 함수
def filter_keywords(df, min_count=10, common_threshold=50):
    '''
    min_count: 최소 등장 횟수
    common_threshold: 공통 키워드의 최소 빈도수
    ex) min_count=10, common_threshold=50 -> 50개 이상의 분류에서 10번 이상 등장한 키워드만 선택
    '''
    # 키워드 분리 및 집계
    df['키워드'] = df['키워드'].str.split(',')
    df_exploded = df.explode('키워드')  # 각 리스트의 요소를 개별 행으로 나눔

    # 분류별 키워드 집계
    keyword_counts = df_exploded.groupby(['분류', '키워드']).size().reset_index(name='빈도수')

    # n개 이상 등장한 키워드 추출
    filtered_keywords = keyword_counts[keyword_counts['빈도수'] >= min_count]

    # 공통 키워드 찾기
    keyword_occurrences = filtered_keywords['키워드'].value_counts()
    common_keywords = keyword_occurrences[keyword_occurrences >= common_threshold].index.tolist()

    # 삭제되는 키워드와 개수
    num_deleted_keywords = len(common_keywords)

    # 삭제되는 키워드의 총 등장 횟수 계산
    total_deleted_occurrences = 0
    for kw in common_keywords:
        total_deleted_occurrences += keyword_occurrences.get(kw, 0)

    # 결과 출력
    print("삭제되는 키워드:", common_keywords)
    print("삭제되는 키워드의 개수(unique): {}".format(num_deleted_keywords))
    print("삭제되는 키워드의 총 개수: {}".format(total_deleted_occurrences))

In [101]:
df1 = train_data.copy()
filter_keywords(df1, 20, 50)

삭제되는 키워드: ['용인시', '경기', '경기도', '서울', '대표', '이날']
삭제되는 키워드의 개수(unique): 6
삭제되는 키워드의 총 개수: 311


In [102]:
df2 = train_data.copy()
filter_keywords(df2, 15, 50)

삭제되는 키워드: ['용인시', '경기', '서울', '경기도', '용인', '상황', '대표', '진행', '이날', '지역']
삭제되는 키워드의 개수(unique): 10
삭제되는 키워드의 총 개수: 517


In [103]:
df3 = train_data.copy()
filter_keywords(df3, 10, 50)

삭제되는 키워드: ['용인시', '경기', '서울', '경기도', '지역', '용인', '시작', '진행', '상황', '관계자', '이날', '마련', '사진', '시민', '대표', '설명', '생각', '참여', '운영', '코로나19', '시장', '예정', '전국', '활동']
삭제되는 키워드의 개수(unique): 24
삭제되는 키워드의 총 개수: 1241


In [104]:
df4 = train_data.copy()
filter_keywords(df4, 15, 45)

삭제되는 키워드: ['용인시', '경기', '서울', '경기도', '용인', '상황', '대표', '진행', '이날', '지역', '관계자', '예정', '시작', '사진', '참여', '활동', '생각', '설명', '코로나19', '시장', '지원', '제공', '전국', '시민', '운영', '한국', '마련', '모습', '시민들', '준비', '가능', '계획', '사업', '위치', '자리', '대상', '확인', '정도', '기록', '사람', '노력']
삭제되는 키워드의 개수(unique): 41
삭제되는 키워드의 총 개수: 1963


In [11]:
# 공통 키워드를 제거하는 함수
def remove_common_keywords(df, common_keywords):
    for index, row in df.iterrows():
        keywords = row['키워드'].split(',')
        # 공통 키워드를 제외한 새로운 키워드 리스트
        filtered_keywords = [kw for kw in keywords if kw not in common_keywords]
        # 필터링된 키워드를 문자열로 다시 합침
        df.at[index, '키워드'] = ', '.join(filtered_keywords)
    return df

In [12]:
common_keywords_list = ['용인시', '경기', '서울', '경기도', '용인', '상황', '대표', '진행', '이날', '지역', '관계자', '예정', '시작', '사진', '참여',
                        '활동', '생각', '설명', '코로나19', '시장', '지원', '제공', '전국', '시민', '운영', '한국', '마련', '모습', '시민들', '준비',
                        '가능', '계획', '사업', '위치', '자리', '대상', '확인', '정도', '기록', '사람', '노력']

train_df = remove_common_keywords(train_data, common_keywords_list)
test_df = remove_common_keywords(test_data, common_keywords_list)

In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
 4   키워드 개수  54609 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
 3   키워드 개수  23405 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 731.5+ KB


In [15]:
# '키워드 개수' 드롭
train_df.drop(columns=['키워드 개수'], inplace=True)
test_df.drop(columns=['키워드 개수'], inplace=True)

### 모델링

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [17]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [20]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [21]:
train_dataset[0]

{'text': '매물, 거래량, 조짐, 집값, 안정, 아시아투데이, 박지숙, 포함, 매물, 수도, 아파트, 거래량, 매수세, 추세, 대세하락, 주택, 안정, 주목, 부동산, 전문가들, 중과, 보유세, 세제, 인상, 매물, 적체, 현상, 공급대책, 관망세, 영향, 정부, 인상, 부동산, 세제, 본격적, 시행, 부담, 다주택자들, 매물, 역대급, 물량, 2, 사전, 청약, 예고, 무주택자들, 매수, 분위기, 해석, 아실, 부동산, 빅데이터, 업체, 아파트실거래가, 매물, 기준, 아파트, 매매, 전세, 월세, 지난달, 11.3%, 7만, 8만, 9만, 이달, 11.7%, 은평구, 이달, 24.2%, 매물, 강남3구, 서초구, 22.2%, 7765건, 9489건, 송파구, 10.1%, 6263건, 6900건, 강남구, 8.8%, 1만, 1만, 강남구, 자치구, 매물, 상태, 안산시, 단원구, 38.7%, 1104건, 1532건, 의왕시, 30.5%, 973건, 1270건, 의정부시, 27.2%, 2301건, 2928건, 강남, 시세, 분당구, 성남시, 4141건, 5195건, 25.4%, 수지구, 4500건, 5386건, 19.7%, 하남시, 23.8%, 2932건, 3616건, 매물, 거래량, 집계, 서울시부동산정보광장, 12월, 3월, 18일, 기준, 거래량, 절반, 추세, 거래량, 7520건, 1월, 3390건, 급감, 3월, 중순, 거래량, 484건, 거래량, 비교, 10분, 수준, 실거래가, 하락, 추세, 부동산114, 강남, 은마아파트, 거래, 115.54, 이달, 23억, 거래, 송파, 헬리오시티, 84.10, 17억, 거래, 17억, 2000만, 매물, 상태, 서초구, 매물, 거래, 보유세, 양도세, 부담, 다주택자들, 연초, 매매가, 분위기, 본부장, 장재현, 리얼투데이, 6월, 보유세, 부담, 매물, 2, 영향, 세제규제, 영향, 다주택자들, 부담, 이태경, 토지, 자유연구소, 부소장, 거래량, 감소, 추세, 매매가, 소득, 극단, 괴리, 30대,

In [22]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 171/171 [01:44<00:00,  1.63it/s]


Validation F1 Score: 0.13


Validating: 100%|██████████| 171/171 [01:45<00:00,  1.62it/s]


Validation F1 Score: 0.27


Validating: 100%|██████████| 171/171 [01:46<00:00,  1.61it/s]


...

Validation F1 Score: 0.47


Validating: 100%|██████████| 171/171 [01:47<00:00,  1.59it/s]


Validation F1 Score: 0.48


Validating: 100%|██████████| 171/171 [01:46<00:00,  1.60it/s]

Validation F1 Score: 0.49


In [24]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [03:46<00:00,  1.61it/s]


In [25]:
sample_submission = pd.read_csv("/content/drive/MyDrive/GBT Hackerton/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/GBT Hackerton/data/submission_KoBert_1003.csv", encoding='UTF-8-sig', index=False)

In [26]:
result = pd.read_csv("/content/drive/MyDrive/GBT Hackerton/data/submission_KoBert_1003.csv")
result.head()

,ID,분류
0,TEST_00000,지역
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:국회_정당
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역


In [27]:
result['분류'].value_counts()

,count
분류,
지역,12086
경제:부동산,1362
사회:사건_사고,1038
경제:반도체,947
사회:교육_시험,502
정치:국회_정당,453
경제:산업_기업,451
경제:취업_창업,439
사회:사회일반,369
